In [1]:
#TASK - 01  Vary the number of qubits (2, 3, 4 qubits)

!pip install qiskit qiskit_aer

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

def run_qft(n):
    qc = qft_circuit(n)
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    statevector = result.data(0)["statevector"]
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

# Run QFT for 2, 3, and 4 qubits
for n_qubits in [2, 3, 4]:
    print(f"\n=== QFT with {n_qubits} qubits ===")
    run_qft(n_qubits)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.8 MB/s eta 0:00:00

=== QFT with 2 qubits ===
                   ┌───┐    statevector 
q_0: ──────■───────┤ H ├─X───────░──────
     ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ┤ H ├─■─────────────X───────░──────
     └───┘                       ░      

=== QFT with 3 qubits ===
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                    

In [2]:
#TASK - 02  Inverse QFT implementation

from qiskit import QuantumCircuit
import numpy as np

def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform."""
    # Reverse the swap operations
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    # Apply inverse rotations
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / 2**(j - m), m, j)
        qc.h(j)
    return qc

# Example: Inverse QFT on 3 qubits
qc_inv = QuantumCircuit(3)
inverse_qft(qc_inv, 3)
qc_inv.name = "Inverse QFT"
print(qc_inv.draw(output='text'))


        ┌───┐                                        
q_0: ─X─┤ H ├─■──────────────■───────────────────────
      │ └───┘ │P(-π/2) ┌───┐ │                       
q_1: ─┼───────■────────┤ H ├─┼─────────■─────────────
      │                └───┘ │P(-π/4)  │P(-π/2) ┌───┐
q_2: ─X──────────────────────■─────────■────────┤ H ├
                                                └───┘


In [3]:
#TASK - 03 Integration with Phase Estimation

from qiskit import QuantumCircuit
import numpy as np

def phase_estimation(unitary, n_count):
    """Phase estimation circuit with QFT."""
    qc = QuantumCircuit(n_count + 1, n_count)

    # Apply Hadamard to counting qubits
    for qubit in range(n_count):
        qc.h(qubit)

    # Apply controlled-U operations
    for qubit in range(n_count):
        qc.cp(2 * np.pi / 2**(n_count - qubit), qubit, n_count)

    # Apply inverse QFT to counting register
    inverse_qft(qc, n_count)
    qc.measure(range(n_count), range(n_count))
    return qc

qc_pe = phase_estimation(None, 3)
qc_pe.name = "Phase Estimation"
print(qc_pe.draw(output='text'))


     ┌───┐                            ┌───┐                                   »
q_0: ┤ H ├─■────────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                        │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■───────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │               │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼────────■──────X──────────────────────■─────────■────────»
     └───┘ │P(π/4)  │P(π/2)  │P(π)                                            »
q_3: ──────■────────■────────■────────────────────────────────────────────────»
                                                                              »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─┐           
«q_0: ┤M├───────────
«     └╥┘     ┌─┐   
«q_1: ─╫──────┤M├───
«      ║ ┌───┐└╥┘┌─┐
«q_2: ─╫─┤ H ├─

In [4]:
#TASK - 04 Measure Output States (simulate probability distribution)


from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

def run_qft_with_measurement(n):
    qc = qft_circuit(n)
    qc.measure_all()
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc.draw(output='text'))

# Run QFT with measurements for 3 qubits
run_qft_with_measurement(3)


                                             ┌───┐    ░ ┌─┐      
   q_0: ──────■──────────────────────■───────┤ H ├─X──░─┤M├──────
              │                ┌───┐ │P(π/2) └───┘ │  ░ └╥┘┌─┐   
   q_1: ──────┼────────■───────┤ H ├─■─────────────┼──░──╫─┤M├───
        ┌───┐ │P(π/4)  │P(π/2) └───┘               │  ░  ║ └╥┘┌─┐
   q_2: ┤ H ├─■────────■───────────────────────────X──░──╫──╫─┤M├
        └───┘                                         ░  ║  ║ └╥┘
meas: 3/═════════════════════════════════════════════════╩══╩══╩═
                                                         0  1  2 


In [5]:
# Task 5: Circuit Visualization
!pip install qiskit qiskit_aer pylatexenc --quiet

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

# QFT helper functions
def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

# --- Build and visualize the QFT circuit ---
qc_vis = qft_circuit(3)   # Change to 2, 3, or 4 qubits
fig = qc_vis.draw(output='mpl')   # Return a matplotlib Figure
plt.show()                 # Force display in Colab/Jupyter

# Also print text version so you always get something visible
print("\nText representation:\n")
print(qc_vis.draw(output='text'))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

Text representation:

                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        
